In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import requests
import json
import pandas as pd
from database import db, cursor
from util import json_load, json_dump
import itertools
from tqdm.notebook import tqdm
import csv
from multiprocessing import Pool
from concurrent import futures
import logging

In [ ]:
url = 'https://kgtk.isi.edu/similarity_api'
headers = {'User-Agent': 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'} 
proxies = {'http': 'http://114.212.82.174:10809', 'https': 'http://114.212.82.174:10809'}
logging.basicConfig(filename='kgtk_similarity.log', level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')


In [ ]:
def call_semantic_similarity(input_file):
    file_name = os.path.basename(input_file)
    files = {
        'file': (file_name, open(input_file, mode='rb'), 'application/octet-stream')
    }
    try:
        resp = requests.post(url, files=files, params={'similarity_types': 'text'}, proxies=proxies, headers=headers)
        s = json.loads(resp.json())
    except Exception as e:
        logging.error(f'Error in calling {url} with {input_file}: {e}')
        s = []
    return s

In [ ]:
from collections import defaultdict
dataset_entity = defaultdict(list)
cursor.execute('SELECT dataset_id, URI FROM `BDR_metadata_link_result_FALCON2` WHERE URI is not NULL;')
for dataset_id, wikidata_uri in cursor.fetchall():
    dataset_id = int(dataset_id)
    wikidata_uri = wikidata_uri.split('/')[-1]
    dataset_entity[dataset_id].append(wikidata_uri)

query_entity = defaultdict(list)
cursor.execute('SELECT query_id, URI FROM `BDR_query_link_result_FALCON2` WHERE URI is not NULL;')
for query_id, wikidata_uri in cursor.fetchall():
    query_id = str(query_id)
    wikidata_uri = wikidata_uri.split('/')[-1]
    query_entity[query_id].append(wikidata_uri)

In [ ]:
# 计算所有可能的pair
pair_set = set()
sparse_res_path = f''
sparse_res = json_load(sparse_res_path)
with tqdm(total=len(sparse_res), ncols=100, leave=True) as pbar:
    for query_id, dataset_id in sparse_res:
        query_entity_list = query_entity[query_id]
        dataset_entity_list = dataset_entity[dataset_id]
        for q, d in itertools.product(query_entity_list, dataset_entity_list):
            pair_set.add((q, d))
    pbar.update(1)
    
with open('kgtk_BDR_pairs.tsv', 'w') as f:
    for pair in pair_set:
        f.write(f'{pair[0]}\t{pair[1]}\n')

In [ ]:
target_list = []
error_list = []
def single_run(unhandled_list=None):
    global target_list
    pair_list = []
    similarity_path = f'/home/xxx/code/reproduce_keds/data/kgtk_similarity/similarity/slices'
    pair_path = f'kgtk_BDR_pairs.tsv'
    for row in csv.reader(open(pair_path, 'r'), delimiter='\t'):
        pair_list.append((row[0], row[1]))
    
    print('pair len:', len(pair_list))
    l = 25

    if unhandled_list:
        for index in unhandled_list:
            index = int(index)
            with open('tmp.tsv', 'w+') as fp:
                fp.write('q1\tq2\n')
                for pair in pair_list[index:index+l]:
                    fp.write(f'{pair[0]}\t{pair[1]}\n')
            try:
                s = call_semantic_similarity('tmp.tsv')
                s = [dict(t) for t in set([tuple(d.items()) for d in s])]
                assert len(s) == l
            except Exception as e:
                logging.error(f'Error in calling {url} with [{index}:{index+l}]: {e} len s: {len(s)}')
                error_list.append(index)
                s = []
            target_list.extend(s)
            logging.info(f'finish {index + l}')
        json_dump(target_list, f'/home/xxx/code/keds/kgtk_similarity/similarity/all_unhandled_similarity.json')
        return
    
    with tqdm(total=len(pair_list), ncols=100, leave=True) as pbar:
        for i in range(110000, len(pair_list), l):
            with open('tmp.tsv', 'w+') as fp:
                fp.write('q1\tq2\n')
                for pair in pair_list[i:i+l]:
                    fp.write(f'{pair[0]}\t{pair[1]}\n')
            try:
                s = call_semantic_similarity('tmp.tsv')
                s = [dict(t) for t in set([tuple(d.items()) for d in s])]
                assert len(s) == len(pair_list[i:i+l])
            except Exception as e:
                logging.error(f'Error in calling {url} with [{i}:{i+l}]: {e} len s: {len(s)}')
                error_list.append(i)
                s = []
            target_list.extend(s)
            logging.info(f'finish {i + l}')
            if int(i + l) % 10000 == 0:
                target_list = [dict(t) for t in set([tuple(d.items()) for d in target_list])]
                json_dump(target_list[i + l - 10000: i + l], f'{similarity_path}/{i + l}.json')
            pbar.update(l)
    target_list = [dict(t) for t in set([tuple(d.items()) for d in target_list])]
    json_dump(target_list, f'/home/xxx/code/reproduce_keds/data/kgtk_similarity/similarity/all_similarity.json')

In [ ]:
single_run()

In [ ]:
all_sim = []
similarity_path = f'/home/xxx/code/reproduce_keds/data/kgtk_similarity/similarity/slices'
for file in os.listdir(similarity_path):
    all_sim.extend(json_load(f'{similarity_path}/{file}'))

json_dump(all_sim, f'/home/xxx/code/reproduce_keds/data/kgtk_similarity/similarity/BDR_all_similarity.json')

In [ ]:
import re

unhandled_list = []
with open('/home/xxx/code/reproduce_keds/kgtk_similarity.log', 'r+') as fp:
    for line in fp.readlines():
        if 'len s:' in line:
            error = re.search(r'\[\d+:\d+\]', line).group(0)
            unhandled_list.append(error[1:-1].split(':')[0])
print(unhandled_list)

In [ ]:
single_run(unhandled_list)

In [ ]:
# src_path = f'/home/xxx/code/keds/kgtk_similarity/similarity/{test_collection}_rank{rank}_similarity.json'
src_path = '/home/xxx/code/keds/kgtk_similarity/similarity/all_similarity.json'

data = json_load(src_path)
row_list = []
for item in data:
    query_entity_uri = 'http://www.wikidata.org/entity/' + item['q1']
    dataset_entity_uri = 'http://www.wikidata.org/entity/' + item['q2']
    query_entity_label = item['q1_label'] if item['q1_label'] else ''
    dataset_entity_label = item['q2_label'] if item['q2_label'] else ''
    text_similarity = item['text'] if item['text'] else -1
    row_list.append((query_entity_uri, dataset_entity_uri, query_entity_label, dataset_entity_label, float(text_similarity)))

# ntcir_cursor_org.executemany(f'INSERT  INTO {test_collection}_query_dataset_entity_similartity_kgtk \
                        #  (query_entity_uri, dataset_entity_uri, query_entity_label, dataset_entity_label, text_similarity ) \
                        #  VALUES (%s,%s,%s,%s,%s);', row_list)
cursor.executemany(f'INSERT  INTO query_dataset_entity_similartity_kgtk \
                         (query_entity_uri, dataset_entity_uri, query_entity_label, dataset_entity_label, text_similarity ) \
                         VALUES (%s,%s,%s,%s,%s);', row_list)

db.commit()